In [1]:
import numpy as np
import pandas as pd

### Implemented hierarchical agglomerative clustering for this dataset. Implemented single (minimum) linkage, complete (maximum) linkage and average (mean) linkage.

In [2]:
#Class of AgglomerativeClustering providing the option to use single, complete and average linkage

class AgglomerativeClustering:
    
    def __init__(self,n_clusters=2,linkage="single"):
        
        self.n_clusters = n_clusters
        self.linkage = linkage

    def fit_predict(self,X):
        
        n=X.shape[0]     
        d=self.d_matrix(X) 
        cluster=self.get_initial_cluster(n)
        s=set(range(n))     
        for _ in range(n-self.n_clusters): 
            p,q=np.unravel_index(np.argmin(d, axis=None), d.shape)
            t_set=s-{p,q} 
            d=self.update_d(d,p,q,t_set,self.linkage) 
            cluster=self.update_cluster(cluster,p,q) 
            s=s-{max(p,q)} 
        decor_l=[]
        for v in cluster.values():
            decor_l.append(v)
        
        self.labels_= self.clustertolabels(decor_l)
        return self.labels_

    def clustertolabels(self,clusters):
        
        ln = sum([len(c) for c in clusters])
        labels = np.zeros(ln,dtype = np.int)
        ind = -1
        for c in clusters:
            ind+=1
            for i in c:
                labels[i] = ind
        return labels


    def d_matrix(self,data):
        
        n=data.shape[0]  
        d=np.empty(shape=[n,n]) 
        d.fill(np.inf)  
        for i in range(n-1):
            for j in range(i+1,n):
                d[i,j]=distance(data[i],data[j]) 
        return d

    
    def get_initial_cluster(self,n):
        
        c={}
        for i in range(n):
            c[i]={i}   
        return c

   
    def update_d(self,d,p,q,t_set,linkage):
        
        for i in t_set: 
            
            u,v=min(i,p),max(i,p) 
            w,x=min(i,q),max(i,q)
            if(linkage=="complete"):
                t=max(d[u,v],d[w,x])
            elif(linkage=="average"):
                t=(d[u,v]+d[w,x])/2
            else:     
                t=min(d[u,v],d[w,x])
        
            d[u,v]=t
            d[w,x]=t
            
        m_pq=max(p,q)
        d[m_pq,:]=np.inf
        d[:,m_pq]=np.inf
        return d


    def update_cluster(self,c,p,q):
        
        i=c.pop(max(p,q)) 
        m=min(p,q)
        c[m]=c[m].union(i) 
        return c

In [3]:
# Distance function calculates the distance using eucledian distance
# Dataset_minmax function calculates the min and max of the dataset
# Normalize_dataset function normalizes the dataset between values 0 and 1

def distance(pt1,pt2):

        if(len(pt1)!=len(pt2)):
            print("Dimensions of the points are not equal")
            return  
        dim=len(pt1)  
        s=0
        for i in range(dim):
            s+=(pt1[i]-pt2[i])**2 
        dist=np.sqrt(s)  
        return dist

def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [4]:
df = pd.read_csv("data2c.csv")
df['Class'].replace([' M', ' W'], [1,0], inplace=True)
labels = df['Class'].values
df = df.iloc[:,:-1]
data = df.values
# Normalized data
minmax = dataset_minmax(data)
normalize_dataset(data, minmax) 

### Applied hierarchical clustering implementation with single (minimum) linkage to the data. Choose the case of 2, 4, 6, and 8 clusters, evaluated to what degree the clusters reflect the gender distinctions in this data by evaluating the purity of the clusters using the original data labels. For this, computed the accuracy for each cluster assuming that the predicted label for each cluster is the most frequently occurring label in this cluster. computed the overall accuracy using the corresponding number of clusters as the weighted sum of the accuracies where the weight is the fraction of data points in the corresponding cluster.

In [5]:
n_clusters = [2,4,6,8]
accuracy_clusters = []
for j in n_clusters:
    clustering = AgglomerativeClustering(n_clusters=j,linkage="single")
    pred_clusters = clustering.fit_predict(data)
    
    weighted_acc = []
    for i in np.unique(pred_clusters):  #Loop for a particular cluster i.e [0,1] if n_clusters = 2  
        indexes = np.where(pred_clusters == i)  #List of indexes where the Clusters predicted has particular value
        arr = [labels[i] for i in indexes]  #Array of the labels w.r.t to the indexes 
        max(np.unique(arr, return_counts=True)[1])  #Maximun of the labels that the data points with particular cluster label has
        accuracy_cluster = max(np.unique(arr, return_counts=True)[1])/len(arr[0])  #Accuracy for each cluster assuming the predicted label for each cluster is most frequently occuring label in the cluster
        weighted_cluster_accuracy = (accuracy_cluster*len(arr[0]))/(len(pred_clusters))
        weighted_acc.append(weighted_cluster_accuracy)
    accuracy_clusters.append(sum(weighted_acc))  #Overall Accuracy using the corresponding number clusters as the weighted sum of the accuracies where weigth is the fraction of data points in the corresponding cluster
    print(f"Accuracy for n = {j} clusters using single linkage : {accuracy_clusters[n_clusters.index(j)]} ")

Accuracy for n = 2 clusters using single linkage : 0.5666666666666667 
Accuracy for n = 4 clusters using single linkage : 0.5666666666666665 
Accuracy for n = 6 clusters using single linkage : 0.5749999999999998 
Accuracy for n = 8 clusters using single linkage : 0.5749999999999998 


### Repeated the clustering and evaluation using complete (maximum) linkage.

In [6]:
n_clusters = [2,4,6,8]
accuracy_clusters = []
for j in n_clusters:
    clustering = AgglomerativeClustering(n_clusters=j,linkage="complete")
    pred_clusters = clustering.fit_predict(data)
    
    weighted_acc = []
    for i in np.unique(pred_clusters):
        indexes = np.where(pred_clusters == i)
        arr = [labels[i] for i in indexes]
        max(np.unique(arr, return_counts=True)[1])
        accuracy_cluster = max(np.unique(arr, return_counts=True)[1])/len(arr[0])
        weighted_cluster_accuracy = (accuracy_cluster*len(arr[0]))/(len(pred_clusters))
        weighted_acc.append(weighted_cluster_accuracy)
    accuracy_clusters.append(sum(weighted_acc))
    print(f"Accuracy for n = {j} clusters using complete linkage : {accuracy_clusters[n_clusters.index(j)]} ")

Accuracy for n = 2 clusters using complete linkage : 0.7749999999999999 
Accuracy for n = 4 clusters using complete linkage : 0.775 
Accuracy for n = 6 clusters using complete linkage : 0.775 
Accuracy for n = 8 clusters using complete linkage : 0.7750000000000001 
